In [2]:
! pip install faiss-cpu tqdm

     ---------------------------------------- 18.2/18.2 MB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 12.9/12.9 MB 4.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
scipy 1.10.0 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.2.6 which is incompatible.
qiskit-ibmq-provider 0.20.2 requires numpy<1.24, but you have numpy 2.2.6 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 2.2.6 which is incompatible.


In [ ]:
import numpy as np
import faiss
from tqdm import tqdm

# Load data
files = ["data/encodings.npy", "data/encodings_2.npy", "data/encodings_3.npy", "data/encodings_4.npy"]
arrays = [np.load(f) for f in files]
data = np.vstack(arrays)
print(f"Data shape: {data.shape}")

N, d = data.shape

# Build a brute-force index in Faiss
index = faiss.IndexFlatL2(d)  # exact L2
index.add(data)  # add all vectors to the index

# Query all vectors for their nearest neighbor
# k=2 because the first neighbor is the point itself
distances, indices = index.search(data, k=2)

# Build dictionary: index -> nearest neighbor (skip self) with tqdm
nn_dict = {}
for i in tqdm(range(N), desc="Building nearest neighbor dictionary"):
    nn_dict[i] = indices[i, 1]  # skip self

# Check first 10
for i in range(10):
    print(f"Vector {i} -> nearest neighbor {nn_dict[i]}")

Data shape: (80000, 384)


Building nearest neighbor dictionary: 100%|██████████| 80000/80000 [00:00<00:00, 1262412.98it/s]

Vector 0 -> nearest neighbor 10215
Vector 1 -> nearest neighbor 58356
Vector 2 -> nearest neighbor 35831
Vector 3 -> nearest neighbor 4
Vector 4 -> nearest neighbor 3
Vector 5 -> nearest neighbor 5210
Vector 6 -> nearest neighbor 14274
Vector 7 -> nearest neighbor 65219
Vector 8 -> nearest neighbor 71475
Vector 9 -> nearest neighbor 5986


In [4]:
import pickle
# Save nn_dict to a file
with open("data/nn_dict.pkl", "wb") as f:
    pickle.dump(nn_dict, f)

print("Nearest neighbor dictionary saved to nn_dict.pkl")

Nearest neighbor dictionary saved to nn_dict.pkl


In [1]:
import numpy as np
import pandas as pd

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject